# Projeto 2 - Ciência dos Dados

Nome: Luiz Felipe Domingues Valente

Nome: João Guilherme Cintra de Freitas Almeida

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [18]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji

In [19]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np
import re 
from emoji import UNICODE_EMOJI

___
## Autenticando no  Twitter

* Conta: @jofenina

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


Capturando os dados do twitter:

Salvando os dados em uma planilha Excel:

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

# Limpeza

In [20]:
tweets_tr_sujos = pd.read_excel("snickers.xlsx", sheet_name = "Treinamento")
tweets_tr_sujos.head()

,Treinamento,Relevância
0,@tiagoatoc nunca pensei que me mandasses comer...,0
1,"credo q eu acordei sem paciência, e já to come...",0
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
4,@yenexiel ninguém compra snickers pra mim quan...,0


In [21]:
treinamento = tweets_tr_sujos.loc[:, "Treinamento"]
rel=tweets_tr_sujos.loc[:, "Relevância"]
treinamento.head()

0    @tiagoatoc nunca pensei que me mandasses comer...
1    credo q eu acordei sem paciência, e já to come...
2    samyra me deu batata frita com cheddar e bacon...
3    crlh acabei de descobrir q tem sorvete de snic...
4    @yenexiel ninguém compra snickers pra mim quan...
Name: Treinamento, dtype: object

## **Criando função de limpeza**

In [22]:
def cleanup(text):
    
#  Limpeza de caracteres

    punctuation = '[!\-.:?;@#$%&*_1234567890]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

## Limpando tweets "Treinamento"

In [23]:
t_limpos1 = []
t_limpos2 = []
t_limpos = []
for linha in treinamento:
    b=linha.lower()
    t_limpos1.append(cleanup(b))
for e in t_limpos1:
    a = e.strip()
    t_limpos2.append(a)
for s in t_limpos2:
    z = ' '.join(s.split())
    t_limpos.append(z)
t_limpos[0:5]

['tiagoatoc nunca pensei que me mandasses comer snickers antes de dormir sinceramente',
 'credo q eu acordei sem paciência, e já to comendo um snickers de café da manhã',
 'samyra me deu batata frita com cheddar e bacon, e evelyn me deu snickers, amo minhas amigas',
 'crlh acabei de descobrir q tem sorvete de snickers eu preciso pra ontem',
 'yenexiel ninguém compra snickers pra mim quando tô triste']

# Espaça emoji

In [24]:
# search your emoji
def is_emoji(s):
    return s in UNICODE_EMOJI

# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

results=[add_space(text) for text in t_limpos]



# Lista de palavras

In [25]:
lista_palavras = []
for palavra in results:
    lista_palavras += palavra.split()


## Frequência de Palavras nos Tweets

In [34]:
serie = pd.Series(lista_palavras)
p_palavras = serie.value_counts(True)
p_palavras.head()


snickers    0.060292
de          0.031581
e           0.026281
um          0.023631
eu          0.021864
dtype: float64

# Tabela com limpeza e relevância

In [27]:
t_tr = pd.DataFrame({"Treinamento": results,"Relevância": rel})
t_tr.head()

,Treinamento,Relevância
0,tiagoatoc nunca pensei que me mandasses comer ...,0
1,"credo q eu acordei sem paciência, e já to come...",0
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
4,yenexiel ninguém compra snickers pra mim quand...,0


# Tweets Relevantes Limpos e frequência de suas Palavras

In [28]:
t_rel= t_tr[(t_tr.Relevância==1)]
t_rel.head()
len(t_rel)

172

In [29]:
serie_tweets = ' '.join(t_rel.loc[:,"Treinamento"])
a = pd.Series(serie_tweets.split())
a.head()

0    samyra
1        me
2       deu
3    batata
4     frita
dtype: object

In [36]:
tabela_tweets_relativar = a.value_counts()
tabela_tweets_relativar.head()

snickers    166
de           92
eu           69
do           62
e            55
dtype: int64

In [33]:
# prob_palavra_relevante = tabela_tweets_relativar / len(lista_palavras)
# prob_palavra_relevante.head()
# prob_palavra_irrelevante = tabela_tweets_relativa_n_rel / len(lista_palavras)
# prob_palavra_irrelevante.head()

NameError: name 'tabela_tweets_relativa_n_rel' is not defined

# Tweets não Relevantes Limpos e frequência de suas Palavras

In [50]:
t_n_rel= t_tr[(t_tr.Relevância==0)]
t_n_rel.head()

,Treinamento,Relevância
0,tiagoatoc nunca pensei que me mandasses comer ...,0
1,"credo q eu acordei sem paciência, e já to come...",0
4,yenexiel ninguém compra snickers pra mim quand...,0
5,se tu roubar uma birosca dentro da favela os c...,0
15,a casa do jovem surtado se resume a pacotes de...,0


In [51]:
serie_tweets_n_rel = ' '.join(t_n_rel.loc[:,"Treinamento"])
a_n_rel = pd.Series(serie_tweets_n_rel.split())
a_n_rel.head()

0    tiagoatoc
1        nunca
2       pensei
3          que
4           me
dtype: object

In [40]:
# Frequencia que a palavra aparece dentro dos irrelevantes
tabela_tweets_relativa_n_rel = a_n_rel.value_counts()
tabela_tweets_relativa_n_rel.head()

snickers    107
e            64
um           54
de           51
o            40
dtype: int64

___
# Função Classificadora

In [56]:
def classificador(tweet):
    palavras=tweet.split()
    prob_r=1
    prob_nr=1
    g = tabela_tweets_relativar.sum()+len(lista_palavras)
    z = tabela_tweets_relativa_n_rel.sum()+len(lista_palavras)
    for p in palavras:
        if p in tabela_tweets_relativar:
            prob_r *= (tabela_tweets_relativar[p]+1) / g
      
        if p in tabela_tweets_relativa_n_rel:
            prob_nr *= (tabela_tweets_relativa_n_rel[p]+1) / z
        else:
            prob_nr *= 1 / z
            prob_r *= 1 / g
    if prob_r > prob_nr:
        return "É relevante"
    else: 
        return "Não é relevante"
print(classificador("eu amo snickers"))

É relevante


___
# Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [44]:
# importando os testes e limpando
tweets_tt_sujos = pd.read_excel("snickers.xlsx", sheet_name = "Teste")
teste = tweets_tt_sujos.loc[:, "Teste"]
relt=tweets_tt_sujos.loc[:, "Relevância"]
tt_limpos1 = []
tt_limpos2 = []
tt_limpos = []
for linhat in teste:
    bt = linhat.lower()
    tt_limpos1.append(cleanup(bt))
for et in tt_limpos1:
    a = et.strip()
    tt_limpos2.append(a)
for st in tt_limpos2:
    z = ' '.join(st.split())
    tt_limpos.append(z)
tt_limpos[0:5]

['hj eu peguei um snickers no veran e paguei',
 'só queria um snickers',
 'q fome, tô com um lanche natural de peito de peru no estômago e um snickers',
 'olukascastro 😧😧😧😧😧 chocada snickers vc vai pagar a dentadura, amada',
 'eu e as meninas acabamos de ganhar snickers de um cliente, amo 💕']

In [45]:
def is_emoji(s):
    return s in UNICODE_EMOJI

# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

resultst=[add_space(text) for text in tt_limpos]
lista_palavrast = []
for pepe in resultst:
    lista_palavrast += pepe.split()
    


In [57]:
serie_te = pd.Series(resultst)
dic_te={}
for frase in serie_te:
    resultado = classificador(frase)
    dic_te[frase]=resultado
ir = 0
inr = 0
for k,v in dic.items():
    if v == 'É relevante':
        ir +=  1
    elif v == 'Não é relevante':
        inr += 1


In [58]:
lista = []
lista2 = []
verdadeiro_positivo = 0
verdadeiro_negativo = 0
falso_positivo = 0
falso_negativo = 0
for k,v in dic.items():
    lista.append(v)

for e in relt:
    lista2.append(e)


In [59]:
lista3=[]
for i in lista:
    if i=="É relevante":
        lista3.append(1)
    if i=="Não é relevante":
        lista3.append(0)
        

In [61]:
i=0
verdadeiro_positivo = 0
verdadeiro_negativo = 0
falso_positivo = 0
falso_negativo = 0
for k in o:
    if k == lista2[i] == 1:
        verdadeiro_positivo += 1
    if k == lista2[i] == 0:
        verdadeiro_negativo += 1
    if k != lista2[i] and lista2[i] == 0:
        falso_positivo += 1
    if k != lista2[i] and lista2[i] == 1:
        falso_negativo += 1
    i += 1
total = falso_positivo+falso_negativo+verdadeiro_positivo+verdadeiro_negativo
        
print(verdadeiro_negativo)
print(verdadeiro_positivo)
print(falso_negativo)
print(falso_positivo)
print(falso_positivo+falso_negativo+verdadeiro_positivo+verdadeiro_negativo)
print('Precisão de acerto dos relevantes segundo nosso código: {}%'.format(((verdadeiro_positivo + falso_negativo)/total)*100))
print('Precisão de acerto dos irrelevantes nosso código: {}%'.format(((verdadeiro_negativo + falso_positivo)/total)*100))

48
34
53
61
196
Precisão de acerto dos relevantes segundo nosso código: 44.38775510204081%
Precisão de acerto dos irrelevantes nosso código: 55.61224489795919%


___
### Concluindo

Conclusão dados:
A análise dos tweets evidenciou que nosso classificador não é muito preciso para classificar tweets sobre o Snickers, que é um chocolate da empresa Mars Wrigley. Nosso classificador teve uma acurácia de aproximadamente 44% de precisão de acerto para tweets relevantes e precisão de aproximadamente 55% de precisão para tweets irrelevantes. Embora a precisão não tenha sido suficiente para usar o algoritmo de forma confiável, ele é um classificador Naive-Bayes, que tem uma base correta e que pode ser refinado e alterado para se obter melhor acurácia.

Possíveis alterações ou incrementações para aumentar a precisão do algoritmo seria: aumentar a base de dados, se pegarmos o dobro, ou o triplo de tweets, é possível que o maior número de palavras para treino aumente a acurácia, também poderia se criar uma função que identifica frases sarcásticas, de tal modo que não classifique certas frases como relevantes assim deixando o classificador mais eficiente. 

O classificador Naive-Bayes pode ser utilizado em outras aplicações, como para analisar a caixa de e-mails de uma pessoa, e distinguindo se certos e-mails vão para a caixa de spam, ou se vão para a caixa principal. Em geral esse método pode ser utilizado para  para otimizar e melhorar softwares ou aparelhos, com o intuito de aumentar a facilidade e a simplicidade de se usar uma ferramenta, assim diminuindo erros e aumentando a produtividade.



## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**